In [1]:
import pandas as pd
import geopandas as gpd
import folium
import json
from shapely.geometry import shape, Point
from bs4 import BeautifulSoup
from folium.plugins import LocateControl, MarkerCluster

# Certificates

In [2]:
epc = pd.read_csv('certificates.csv')
epc.head()

,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
0,e2312659dbfb01ce5624ad673bec8b5300fee74eec5188...,2 Cloverfield Avenue,NaN,NaN,NE3 3NJ,10001401826,D,C,67,80,...,Newcastle upon Tyne,Newcastle upon Tyne North,NEWCASTLE UPON TYNE,England and Wales: 1930-1949,2021-09-30 07:47:38,Owner-occupied,10.0,NaN,4.510057e+09,Energy Assessor
1,05b12a8b824957bd19efdb0f37709271f8a9064b86881c...,"11, Primrose Lane",NaN,NaN,NE13 9EU,10001346230,B,A,84,95,...,Newcastle upon Tyne,Newcastle upon Tyne North,NEWCASTLE UPON TYNE,2020,2021-09-30 06:57:34,Owner-occupied,15.0,NaN,NaN,NaN
2,17b2af8aa808b7bcc1a3c15c0f0c32a496ffb6c0a643c4...,9 Woburn Way,NaN,NaN,NE5 5JD,10001561521,D,C,63,79,...,Newcastle upon Tyne,Newcastle upon Tyne North,NEWCASTLE UPON TYNE,England and Wales: 1967-1975,2021-09-30 13:57:45,Owner-occupied,10.0,NaN,4.510049e+09,Energy Assessor
3,1a53ae93a4644388b08f84694948c49120a6cba4c5b18a...,49 MacDonald Road,NaN,NaN,NE4 8XU,10001514661,D,C,62,72,...,Newcastle upon Tyne,Newcastle upon Tyne Central,NEWCASTLE UPON TYNE,England and Wales: 1930-1949,2021-09-30 14:31:38,Rented (private),8.0,NaN,4.510085e+09,Energy Assessor
4,0d4602b8f04902aba816f4590ea2e7ec33e286782f3546...,91 Mary Magdalene Bungalows,Claremont Road,NaN,NE2 4NN,10001562311,D,B,67,85,...,Newcastle upon Tyne,Newcastle upon Tyne Central,NEWCASTLE UPON TYNE,England and Wales: 1950-1966,2021-09-30 09:45:52,Rented (private),5.0,NaN,4.510142e+09,Energy Assessor


In [3]:
post = pd.read_csv('ukpostcodes.csv')
post.head()

,id,postcode,latitude,longitude
0,1,AB10 1XG,57.144165,-2.114848
1,2,AB10 6RN,57.137880,-2.121487
2,3,AB10 7JB,57.124274,-2.127190
3,4,AB11 5QN,57.142701,-2.093295
4,5,AB11 6UL,57.137547,-2.112233


In [4]:
post["Code_is_newc"] = post["postcode"].str.startswith('NE')
post

,id,postcode,latitude,longitude,Code_is_newc
0,1,AB10 1XG,57.144165,-2.114848,False
1,2,AB10 6RN,57.137880,-2.121487,False
2,3,AB10 7JB,57.124274,-2.127190,False
3,4,AB11 5QN,57.142701,-2.093295,False
4,5,AB11 6UL,57.137547,-2.112233,False
...,...,...,...,...,...
1768721,2686324,YO7 3FQ,54.223396,-1.357010,False
1768722,2686325,YO7 3FR,54.223853,-1.356849,False
1768723,2686326,YO8 1DX,53.784954,-1.067704,False
1768724,2686327,YO8 5DL,53.838238,-0.996403,False


In [5]:
post2 = post[post['Code_is_newc'] == True]
post3 = post2[['postcode', 'latitude', 'longitude']]
post3

,postcode,latitude,longitude
675070,NE99 2BL,54.967367,-1.615383
675071,NE99 2BN,54.967467,-1.615396
675072,NE99 2DE,54.967367,-1.615383
675073,NE99 2PT,54.967367,-1.615383
675074,NE99 2UT,54.967367,-1.615383
...,...,...,...
1767004,NE8 9FP,54.958280,-1.600648
1767005,NE9 9DU,54.933196,-1.610588
1767006,NE9 9DW,54.933196,-1.610588
1767007,NE99 2GR,54.967467,-1.615396


In [6]:
epc2 = epc[['ADDRESS1', 'POSTCODE', 'CURRENT_ENERGY_RATING', 'POTENTIAL_ENERGY_RATING']]
epc2 = epc2.rename(columns ={'POSTCODE': 'postcode'})
epc2.head()

,ADDRESS1,postcode,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING
0,2 Cloverfield Avenue,NE3 3NJ,D,C
1,"11, Primrose Lane",NE13 9EU,B,A
2,9 Woburn Way,NE5 5JD,D,C
3,49 MacDonald Road,NE4 8XU,D,C
4,91 Mary Magdalene Bungalows,NE2 4NN,D,B


In [7]:
epcd = pd.merge(post3, epc2, on = 'postcode')
epcd.head()

,postcode,latitude,longitude,ADDRESS1,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING
0,NE7 7HX,54.994677,-1.584785,21 WELDON CRESCENT,D,C
1,NE7 7JA,54.996002,-1.583411,70 WELDON CRESCENT,D,C
2,NE7 7JD,54.996551,-1.586250,126 Weldon Crescent,C,B
3,NE7 7JR,54.994109,-1.574037,24 AMBERLEY GARDENS,C,B
4,NE7 7JU,54.995786,-1.572863,51 COCHRANE PARK AVENUE,D,B


In [8]:
epc_dict = epcd.set_index('ADDRESS1').stack().groupby(level=0).agg(list).to_dict()
epc_dict

{' 11 Maxwell Court': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 ' 152 WESTGATE COURT': ['NE4 6HA', 54.972686, -1.632033, 'D', 'C'],
 ' 21': ['NE6 4AZ', 54.980149, -1.562964, 'B', 'B'],
 ' 24 WOLSELEY GARDENS': ['NE2 1HR', 54.985674, -1.590472, 'C', 'C'],
 ' 30 EASTFIELD HOUSE': ['NE6 4UR', 54.985401, -1.541782, 'D', 'C'],
 ' 4 MAXWELL COURT': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 ' 5 Maxwell Court': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 ' 6 Maxwell Court': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 ' 67 EASTFIELD HOUSE': ['NE6 4US', 54.985401, -1.541782, 'D', 'C'],
 ' 7 MAXWELL COURT': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 ' 8 Maxwell Court': ['NE15 6NZ', 54.975082, -1.673458, 'C', 'C'],
 ' 9 Maxwell Court': ['NE15 6NZ', 54.975082, -1.673458, 'B', 'B'],
 '1 AINDERBY ROAD': ['NE15 9DQ', 54.996458, -1.76818, 'D', 'C'],
 '1 ALNHAM COURT': ['NE3 2JT', 55.015734, -1.654893, 'D', 'C'],
 '1 Aln Court': ['NE15 8ER', 54.979008, -1.717908, 'D', 'B'],
 '1 Amble

In [9]:
ncl = gpd.read_file('LAD_DEC_2021_UK_BFC.shp')

In [10]:
# Create the map
NewC = folium.Map(location=[55.01, -1.64], tiles='OpenStreetMap', zoom_start=11.3, control_scale=True, prefer_canvas=True)
folium.GeoJson(data=ncl["geometry"]).add_to(NewC)

In [ ]:
%%time


for address, details in epc_dict.items():
    # Define marker variables
    name = address
    latitude = details[1]
    longitude = details[2]
    postcode = details[0]
    current_energy = details[3]
    potential = details[4]
    
    html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;">{name}</span></p>
    <p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 12px;">{name}'s postcode: {postcode}</span></p>
    <p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 12px;">Current energy rating: {current_energy}</span></p>
    <p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 12px;">Potential energy rating: {potential}</span></p>
    """, script=True)

    iframe = folium.IFrame(html, width=200, height=200)

    popup = folium.Popup(iframe, max_width=200)

    cluster = MarkerCluster(show=True)
    
    folium.Marker(location = [latitude, longitude], popup=popup).add_to(cluster)
    
    cluster.add_to(NewC)
        
NewC

Wall time: 1.44 s


In [ ]:
outfp = "Energy_efficency.html"
NewC.save(outfp)